## Import necessary modules
Run this cell before running any other cells

In [2]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np

LOG.propagate = False

# Printing and Logging
## Printing
You can use the **print()** function in Python to print messages to the screen. <br>
The message can be a string, or any other object, the object will be converted into a string before it is written to the screen. <br>

## Logging
You could use the logging module that is setup in *utils.py*. <br>
It prints to both your screen (standard output) as well as to log files (*ble.log*) in the *logs* directory. <br>
This is the recommended way to output messages, since the log files can help with debugging. <br>
The logging module also provides different log levels as shown below, each formatted with a different color for increased visibility. <br>

__**NOTE**__: You may notice that the DEBUG message is not printed to the screen but is printed in the log file. This is because the logging level for the screen is set to INFO and for the file is set to DEBUG. You can change the default log levels in *utils.py* (**STREAM_LOG_LEVEL** and **FILE_LOG_LEVEL**). 

## Formatting output
To format your strings, you may use %-formatting, str.format() or f-strings. <br>
The most "pythonic" way would be to use f-strings. [Here](https://realpython.com/python-f-strings/) is a good tutorial on f-strings. <br>

In [3]:
LOG.debug("debug")
LOG.info("info")
LOG.warning("warning")
LOG.error("error")
LOG.critical("critical")

2022-02-07 20:11:31,150 | INFO     |: info
2022-02-07 20:11:31,152 | WARNING  |: warning
2022-02-07 20:11:31,153 | ERROR    |: error
2022-02-07 20:11:31,155 | CRITICAL |: critical


<hr>

# BLE
## ArtemisBLEController
The class **ArtemisBLEController** (defined in *ble.py*) provides member functions to handle various BLE operations to send and receive data to/from the Artemis board, provided the accompanying Arduino sketch is running on the Artemis board. <br>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Functions</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">reload_config()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Reload changes made in <em>connection.yaml.</em></span></th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">connect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Connect to the Artemis board, whose MAC address is specified in <em>connection.yaml</em>.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">disconnect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Disconnect from the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">is_connected()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Return a boolean indicating whether your controller is connected to the Artemis board or not.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">send_command(cmd_type, data)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Send the command <strong>cmd_type</strong> (integer) with <strong>data</strong> (string) to the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">receive_float(uuid) <br> receive_string(uuid) <br> receive_int(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Read the GATT characteristic (specified by its <strong>uuid</strong>) of type float, string or int. <br> The type of the GATT
            characteristic is determined by the classes BLEFloatCharacteristic, BLECStringCharacteristic or
            BLEIntCharacteristic in the Arduino sketch.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">start_notify(uuid, notification_handler)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Activate notifications on the GATT characteristic (specified by its <strong>uuid</strong>). <br> <strong>notification_handler</strong> is a
            function callback which must accept two inputs; the first will be a uuid string object and the second will
            be the bytearray of the characteristic value.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">bytearray_to_float(byte_array) <br> bytearray_to_string(byte_array) <br> bytearray_to_int(byte_array)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Convert the <strong>bytearray</strong> to float, string or int, respectively. <br> You may use these functions inside your
            notification callback function.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">stop_notify(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Stop notifications on the GATT characteristic (specified by its <strong>uuid</strong>).</span></th>
    </tr>
</table>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Variables</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">uuid</span></th>
        <th style="text-align: left"><span style="font-weight: normal">A dictionary that stores the UUIDs of the various characteristics specified in <em>connection.yaml</em>.</span></th>
    </tr>
</table>

## Configuration
- The MAC address, Service UUID and GATT characteristic UUIDs are defined in the file: *connection.yaml*.
- They should match the UUIDs used in the Arduino sketch.
- The artemis board running the base code should display its MAC address in the serial monitor.
- Update the **artemis_address** in *connection.yaml*, accordingly.
- Make sure to call **ble.reload_config()** or **get_ble_controller()** (which internally calls **reload_config()**) after making any changes to your configuration file.

<hr>

In the below cell, we create an **ArtemisBLEController** object using **get_ble_controller()** (defined in *ble.py*), which creates and/or returns a single instance of **ArtemisBLEController**. <br>
<span style="color:rgb(240,50,50)"> __NOTE__: Do not use the class directly to instantiate an object. </span><br>

In [18]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2022-02-07 20:26:57,204 | INFO     |: Looking for Artemis Nano Peripheral Device: C0:83:B0:6C:9A:3C
2022-02-07 20:26:59,681 | INFO     |: Connected to C0:83:B0:6C:9A:3C


## Receive data from the Artemis board

The cell below shows examples of reading different types (as defined in the Arduino sketch) of GATT characteristics.

In [ ]:
# Read a float GATT Charactersistic
f = ble.receive_float(ble.uuid['RX_FLOAT'])
print(f)

In [ ]:
# Read a string GATT Charactersistic
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

## Send a command to the Artemis board
Send the PING command and read the reply string from the string characteristic RX_STRING. <br>
__NOTE__: The **send_command()** essentially sends a string data to the GATT characteristic (TX_CMD_STRING). The GATT characteristic in the Arduino sketch is of type BLECStringCharacteristic.

In [ ]:
ble.send_command(CMD.PING, "")

In [ ]:
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

The cell below shows an example of the SEND_TWO_INTS command. <br> The two values in the **data** are separated by a delimiter "|". <br>
Refer Lab 2 documentation for more information on the command protocol.

In [ ]:
ble.send_command(CMD.SEND_TWO_INTS, "2|-6")

The Artemis board should print the two integers to the serial monitor in the ArduinoIDE. 

In [ ]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

In [ ]:
ble.send_command(CMD.ECHO, "HiHello")

s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

In [ ]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

In [ ]:
ble.send_command(CMD.SEND_THREE_FLOATS, "1.4|2.5|-1.7")

In [19]:
received_float = 0

def receive_float_notif_handler(uuid, byte_array_value):
    global received_float, values
    received_float = ble.bytearray_to_float(byte_array_value)
    print(received_float)
    return received_float

In [20]:
ble.start_notify(ble.uuid['RX_FLOAT'], receive_float_notif_handler)

start_time = time.time()
while(time.time() - start_time < 2):
     time.sleep(1)

ble.stop_notify(ble.uuid['RX_FLOAT'])

4394.0
4394.5
4395.0
4395.5
4396.0
4396.5
4397.0
4397.5
4398.0
4398.5
4399.0
4399.5
4400.0
4400.5
4401.0
4401.5
4402.0
4402.5
4403.0
4403.5
4404.0
4404.5
4405.0
4405.5
4406.0
4406.5
4407.0
4407.5
4408.0
4408.5
4409.0
4409.5
4410.0
4410.5
4411.0
4411.5
4412.0
4412.5
4413.0
4413.5
4414.0
4414.5
4415.0
4415.5
4416.0
4416.5
4417.0
4417.5
4418.0
4418.5
4419.0
4419.5
4420.0
4420.5
4421.0
4421.5
4422.0
4422.5
4423.0
4423.5
4424.0
4424.5
4425.0
4425.5
4426.0
4426.5
4427.0
4427.5
4428.0
4428.5
4429.0
4429.5
4430.0
4430.5
4431.0
4431.5
4432.0
4432.5
4433.0
4433.5
4434.0
4434.5
4435.0
4435.5
4436.0
4436.5
4437.0
4437.5
4438.0
4438.5
4439.0
4439.5
4440.0
4440.5
4441.0
4441.5
4442.0
4442.5
4443.0
4443.5
4444.0
4444.5
4445.0
4445.5
4446.0
4446.5
4447.0
4447.5
4448.0
4448.5
4449.0
4449.5
4450.0
4450.5
4451.0
4451.5
4452.0
4452.5
4453.0
4453.5
4454.0
4454.5
4455.0
4455.5
4456.0
4456.5
4457.0
4457.5
4458.0
4458.5
4459.0
4459.5
4460.0
4460.5
4461.0
4461.5
4462.0
4462.5
4463.0
4463.5
4464.0
4464.5
4465.0

In [ ]:
from sys import getsizeof

In [ ]:
messages = ["", " ", "hello", "hello hello hello"] # send messages of varying sizes
data_sizes = []
data_rates = []

for message in messages:
    
    message_times = []
    for i in range(10): # average the results of ten trials
        start_time = time.time()
        ble.send_command(CMD.ECHO, message)
        response = ble.receive_string(ble.uuid['RX_STRING'])
        time_elapsed = time.time() - start_time
        time.sleep(1) # prevent dropped packets
        
        message_times.append(time_elapsed)
    
    data_size = getsizeof(message) + getsizeof(response) # in bytes
    data_sizes.append(data_size)
    
    data_rate = data_size / np.mean(message_times)
    data_rates.append(data_rate)

In [ ]:
import matplotlib.pyplot as plt

print(data_sizes)
print(data_rates)

In [ ]:
plt.plot(data_sizes, data_rates)
plt.xlabel("Data Size (bytes)")
plt.ylabel("Data Rate (bytes/sec)")
plt.title("Data Rates for Various Message Sizes")
plt.savefig("dataRate.png")
plt.show()

In [ ]:
start_time = time.time()
data = ble.receive_float(ble.uuid['RX_FLOAT'])
data_size = getsizeof(data) # in bytes
elapsed_time = time.time() - start_time
print(data_size/elapsed_time)

## Disconnect

In [ ]:
# Disconnect
ble.disconnect()